# 01 MSE monitoring

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq
from sklearn.metrics import mean_squared_error

In [2]:
os.chdir('../..')

In [5]:
from src.data.s3_utilities import s3_filesystem

In [6]:
pd.options.display.max_columns = None

## MSE

In [7]:
gameweek_1_predictions = pq.read_table('s3://fpl-analysis-data/gw_predictions/season=2020-21/gw=1/2020-21-1.parquet', filesystem=s3_filesystem).to_pandas()
print(gameweek_1_predictions.shape)
gameweek_1_predictions.head()

(655, 39)


,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model
0,5.657178,5.443412,5.453287,5.351354,5.367841,27.273075,mohamed_salah,0.0,0.0,0.0,1.0,Liverpool,12.0,mohamed_salah,5.054811,5.089550,5.205740,5.161764,5.029424,25.541290,mohamed_salah,0.0,0.0,0.0,1.0,mohamed_salah,mohamed_salah,both,5.054811,5.089550,5.205740,5.161764,5.029424,25.541290,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
1,6.319860,6.077927,6.090129,5.983685,6.004195,30.475794,raheem_sterling,0.0,0.0,0.0,1.0,Manchester City,11.5,raheem_sterling,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,raheem_sterling,0.0,0.0,0.0,1.0,raheem_sterling,raheem_sterling,both,4.751492,5.080462,4.870038,4.858727,4.749552,24.310270,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
2,4.679602,4.493706,4.491284,4.391965,4.398650,22.455206,harry_kane,0.0,1.0,0.0,0.0,Tottenham Hotspur,10.5,harry_kane,4.633261,4.662464,4.680997,4.614158,4.518755,23.109636,harry_kane,0.0,1.0,0.0,0.0,harry_kane,harry_kane,both,4.633261,4.662464,4.680997,4.614158,4.518755,23.109636,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
3,4.239182,4.072062,4.067756,3.971863,3.975711,20.326572,andrew_robertson,1.0,0.0,0.0,0.0,Liverpool,7.0,andrew_robertson,5.063799,4.396431,4.407769,4.238287,4.086255,22.192539,andrew_robertson,1.0,0.0,0.0,0.0,andrew_robertson,andrew_robertson,both,5.063799,4.396431,4.407769,4.238287,4.086255,22.192539,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01
4,4.516146,4.338603,4.335416,4.238196,4.243636,21.671997,pierre-emerick_aubameyang,0.0,1.0,0.0,0.0,Arsenal,12.0,pierre-emerick_aubameyang,4.204607,4.468777,4.324972,4.306936,4.172596,21.477888,pierre-emerick_aubameyang,0.0,0.0,0.0,1.0,pierre-emerick_aubameyang,pierre-emerick_aubameyang,both,4.204607,4.468777,4.324972,4.306936,4.172596,21.477888,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01


In [8]:
gameweek_actuals = pd.read_parquet('data/gw_player_data/gw_1_player_data.parquet')
print(gameweek_actuals.shape)
gameweek_actuals.head()

(425, 44)


,total_points,was_home,team_h_score,team_a_score,gw,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,name,team_name,promoted_side,top_6_last_season,season,position_DEF,position_FWD,position_GK,position_MID,team_name_opponent,promoted_side_opponent,top_6_last_season_opponent,team_name_ffs,kickoff_month_Sep,late_kickoff,early_kickoff,ID
0,0.0,False,0.0,3.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,76656.0,0.0,0.0,mesut_özil,Arsenal,0,0,2020-21,0,0,0,1,Fulham,1,0,Fulham,1,0,1,294
1,0.0,False,0.0,3.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,12184.0,0.0,0.0,sokratis_papastathopoulos,Arsenal,0,0,2020-21,1,0,0,0,Fulham,1,0,Fulham,1,0,1,381
2,0.0,False,0.0,3.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,0.0,69564.0,0.0,0.0,david_luiz moreira marinho,Arsenal,0,0,2020-21,1,0,0,0,Fulham,1,0,Fulham,1,0,1,95
3,7.0,False,0.0,3.0,1,90.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,19.0,36.6,15.3,54.0,10.6,12.0,0.0,2823465.0,0.0,0.0,pierre-emerick_aubameyang,Arsenal,0,0,2020-21,0,0,0,1,Fulham,1,0,Fulham,1,0,1,338
4,0.0,False,0.0,3.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,35774.0,0.0,0.0,cédric_soares,Arsenal,0,0,2020-21,1,0,0,0,Fulham,1,0,Fulham,1,0,1,81


In [9]:
points_comparison = gameweek_1_predictions.merge(
    gameweek_actuals[['name', 'total_points']], 
    left_on='name_old', 
    right_on='name', 
    how='inner', 
    suffixes=('', '_actuals')
)

In [10]:
points_comparison[points_comparison['sum'].isnull()]

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model,name_actuals,total_points
315,NaN,NaN,NaN,NaN,NaN,NaN,fraser_forster,0.0,0.0,1.0,0.0,Southampton,NaN,fraser_forster,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,lstm_v4,fraser_forster,0.0
316,NaN,NaN,NaN,NaN,NaN,NaN,matt_macey,0.0,0.0,1.0,0.0,Arsenal,NaN,matt_macey,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,lstm_v4,matt_macey,0.0
317,NaN,NaN,NaN,NaN,NaN,NaN,sam_woods,1.0,0.0,0.0,0.0,Crystal Palace,NaN,sam_woods,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,lstm_v4,sam_woods,0.0


In [11]:
points_comparison[points_comparison['sum_old'].isnull()]

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model,name_actuals,total_points
256,NaN,NaN,NaN,NaN,NaN,NaN,phil_jagielka,1.0,0.0,0.0,0.0,Sheffield United,NaN,phil_jagielka,0.308877,0.435307,0.520148,0.568090,0.607233,2.439655,phil_jagielka,1.0,0.0,0.0,0.0,phil_jagielka,phil_jagielka,both,0.308877,0.435307,0.520148,0.568090,0.607233,2.439655,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01,phil_jagielka,0.0
287,NaN,NaN,NaN,NaN,NaN,NaN,nathan_tella,0.0,0.0,0.0,1.0,Southampton,NaN,nathan_tella,0.178753,0.322427,0.368829,0.438472,0.469501,1.777982,nathan_tella,0.0,0.0,0.0,1.0,nathan_tella,nathan_tella,both,0.178753,0.322427,0.368829,0.438472,0.469501,1.777982,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01,nathan_tella,0.0
315,NaN,NaN,NaN,NaN,NaN,NaN,fraser_forster,0.0,0.0,1.0,0.0,Southampton,NaN,fraser_forster,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,lstm_v4,fraser_forster,0.0
316,NaN,NaN,NaN,NaN,NaN,NaN,matt_macey,0.0,0.0,1.0,0.0,Arsenal,NaN,matt_macey,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,lstm_v4,matt_macey,0.0
317,NaN,NaN,NaN,NaN,NaN,NaN,sam_woods,1.0,0.0,0.0,0.0,Crystal Palace,NaN,sam_woods,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,left_only,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,lstm_v4,sam_woods,0.0


In [12]:
points_comparison = points_comparison[~points_comparison['sum'].isnull()]
points_comparison = points_comparison[~points_comparison['sum_old'].isnull()]

In [13]:
points_comparison.head()

,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model,name_actuals,total_points
0,5.657178,5.443412,5.453287,5.351354,5.367841,27.273075,mohamed_salah,0.0,0.0,0.0,1.0,Liverpool,12.0,mohamed_salah,5.054811,5.089550,5.205740,5.161764,5.029424,25.541290,mohamed_salah,0.0,0.0,0.0,1.0,mohamed_salah,mohamed_salah,both,5.054811,5.089550,5.205740,5.161764,5.029424,25.541290,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01,mohamed_salah,20.0
1,4.679602,4.493706,4.491284,4.391965,4.398650,22.455206,harry_kane,0.0,1.0,0.0,0.0,Tottenham Hotspur,10.5,harry_kane,4.633261,4.662464,4.680997,4.614158,4.518755,23.109636,harry_kane,0.0,1.0,0.0,0.0,harry_kane,harry_kane,both,4.633261,4.662464,4.680997,4.614158,4.518755,23.109636,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01,harry_kane,2.0
2,4.239182,4.072062,4.067756,3.971863,3.975711,20.326572,andrew_robertson,1.0,0.0,0.0,0.0,Liverpool,7.0,andrew_robertson,5.063799,4.396431,4.407769,4.238287,4.086255,22.192539,andrew_robertson,1.0,0.0,0.0,0.0,andrew_robertson,andrew_robertson,both,5.063799,4.396431,4.407769,4.238287,4.086255,22.192539,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01,andrew_robertson,4.0
3,4.516146,4.338603,4.335416,4.238196,4.243636,21.671997,pierre-emerick_aubameyang,0.0,1.0,0.0,0.0,Arsenal,12.0,pierre-emerick_aubameyang,4.204607,4.468777,4.324972,4.306936,4.172596,21.477888,pierre-emerick_aubameyang,0.0,0.0,0.0,1.0,pierre-emerick_aubameyang,pierre-emerick_aubameyang,both,4.204607,4.468777,4.324972,4.306936,4.172596,21.477888,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01,pierre-emerick_aubameyang,7.0
4,3.004309,2.883254,2.870653,2.782548,2.777399,14.318163,hugo_lloris,0.0,0.0,1.0,0.0,Tottenham Hotspur,5.5,hugo_lloris,4.482298,4.335516,4.307123,4.188541,4.082161,21.395639,hugo_lloris,0.0,0.0,1.0,0.0,hugo_lloris,hugo_lloris,both,4.482298,4.335516,4.307123,4.188541,4.082161,21.395639,0.0,0.0,1.0,0.0,DeepFantasyFootball_v01,hugo_lloris,3.0


In [14]:
NUM_TRIALS = 1000

In [15]:
# MSE for used score
mse_list = []
for trial in range(NUM_TRIALS):
    sample_df = points_comparison.sample(n=100)
    mse = mean_squared_error(sample_df['GW_plus_1'], sample_df['total_points'])
    mse_list.append(mse)

np.mean(mse_list), np.std(mse_list)

(7.549912414183283, 1.987190357913961)

In [16]:
# MSE for DeepFantasyFootball score
mse_list = []
for trial in range(NUM_TRIALS):
    sample_df = points_comparison[points_comparison['model'] == 'DeepFantasyFootball_v01'].sample(n=100)
    mse = mean_squared_error(sample_df['GW_plus_1'], sample_df['total_points'])
    mse_list.append(mse)

np.mean(mse_list), np.std(mse_list)

(7.983867772788564, 2.036490989165956)

In [17]:
# MSE for LSTM_v4 score
mse_list = []
for trial in range(NUM_TRIALS):
    sample_df = points_comparison[points_comparison['model'] == 'DeepFantasyFootball_v01'].sample(n=100)
    mse = mean_squared_error(sample_df['GW_plus_1_old'], sample_df['total_points'])
    mse_list.append(mse)

np.mean(mse_list), np.std(mse_list)

(7.666307154074613, 2.0356268361411884)

Some evidence that new model may be worse for gameweek 1. Let's see how it does over the coming gameweeks.

## Prediction changes

In [18]:
gameweek_2_predictions = pq.read_table('s3://fpl-analysis-data/gw_predictions/season=2020-21/gw=2/2020-21-2.parquet', filesystem=s3_filesystem).to_pandas()
print(gameweek_2_predictions.shape)
gameweek_2_predictions.head()

(479, 39)


,GW_plus_1_old,GW_plus_2_old,GW_plus_3_old,GW_plus_4_old,GW_plus_5_old,sum_old,name_old,position_DEF_old,position_FWD_old,position_GK_old,position_MID_old,team_name,next_match_value,name,GW_plus_1_new,GW_plus_2_new,GW_plus_3_new,GW_plus_4_new,GW_plus_5_new,sum_new,name_new,position_DEF_new,position_FWD_new,position_GK_new,position_MID_new,name_formatted_new,name_join_key,_merge,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,position_DEF,position_FWD,position_GK,position_MID,model
0,6.369579,6.127182,6.139789,6.033851,6.054636,30.725039,mohamed_salah,0.0,0.0,0.0,1.0,Liverpool,12.0,mohamed_salah,5.230058,5.658565,5.410153,5.408102,5.164496,26.871374,mohamed_salah,0.0,0.0,0.0,1.0,mohamed_salah,mohamed_salah,both,5.230058,5.658565,5.410153,5.408102,5.164496,26.871374,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
1,6.294118,6.053049,6.065045,5.958680,5.978988,30.349880,raheem_sterling,0.0,0.0,0.0,1.0,Manchester City,11.5,raheem_sterling,4.865597,4.983872,4.781246,4.754177,4.660809,24.045700,raheem_sterling,0.0,0.0,0.0,1.0,raheem_sterling,raheem_sterling,both,4.865597,4.983872,4.781246,4.754177,4.660809,24.045700,0.0,0.0,0.0,1.0,DeepFantasyFootball_v01
2,4.338333,4.166255,4.162456,4.065246,4.069820,20.802111,harry_kane,0.0,1.0,0.0,0.0,Tottenham Hotspur,10.5,harry_kane,4.231596,4.734409,4.638270,4.685783,4.507912,22.797970,harry_kane,0.0,1.0,0.0,0.0,harry_kane,harry_kane,both,4.231596,4.734409,4.638270,4.685783,4.507912,22.797970,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
3,4.724390,4.537116,4.535508,4.436080,4.443346,22.676439,jamie_vardy,0.0,1.0,0.0,0.0,Leicester City,10.0,jamie_vardy,4.969102,4.381284,4.326105,4.194391,4.133353,22.004234,jamie_vardy,0.0,1.0,0.0,0.0,jamie_vardy,jamie_vardy,both,4.969102,4.381284,4.326105,4.194391,4.133353,22.004234,0.0,1.0,0.0,0.0,DeepFantasyFootball_v01
4,3.974030,3.816989,3.811089,3.716642,3.718597,19.037348,andrew_robertson,1.0,0.0,0.0,0.0,Liverpool,7.0,andrew_robertson,4.051450,4.595518,4.469839,4.486474,4.315575,21.918856,andrew_robertson,1.0,0.0,0.0,0.0,andrew_robertson,andrew_robertson,both,4.051450,4.595518,4.469839,4.486474,4.315575,21.918856,1.0,0.0,0.0,0.0,DeepFantasyFootball_v01


In [19]:
gameweek_1_predictions['model'].value_counts()

DeepFantasyFootball_v01    424
lstm_v4                    231
Name: model, dtype: int64

In [20]:
gameweek_2_predictions['model'].value_counts()

DeepFantasyFootball_v01    276
lstm_v4                    203
Name: model, dtype: int64

Drop off in number of predictions probably makes sense because of relegated teams no longer in the PL and teams who have been promoted and are yet to play 5 gameweeks (minimum required to make predictions)

In [31]:
combined = gameweek_1_predictions.merge(gameweek_2_predictions, on='name', how='inner', suffixes=('_GW1', '_GW2'))

In [32]:
combined['rank_GW1'] = combined['sum_GW1'].rank(ascending=False)
combined['rank_GW2'] = combined['sum_GW2'].rank(ascending=False)

In [33]:
combined.sort_values('sum_GW2', ascending=False, inplace=True)

In [34]:
combined['rank_increase'] = combined['rank_GW1'] - combined['rank_GW2']

In [37]:
combined[['name', 'rank_increase']].head(20)

,name,rank_increase
0,mohamed_salah,0.0
1,raheem_sterling,0.0
2,harry_kane,0.0
10,jamie_vardy,7.0
3,andrew_robertson,-1.0
7,kevin_de_bruyne,2.0
9,gabriel_fernando_de_jesus,3.0
5,ederson_santana_de_moraes,-2.0
21,sadio_mane,13.0
11,bruno_miguel_borges_fernandes,2.0


In [38]:
combined.sort_values('rank_increase')[['name', 'rank_increase']].head(20)

,name,rank_increase
86,christian_benteke,-215.0
81,neco_williams,-169.0
107,willy_caballero,-160.0
58,edward_nketiah,-149.0
274,oliver_skipp,-123.0
290,billy_gilmour,-118.0
16,emiliano_martinez,-116.0
101,demarai_gray,-115.0
150,kelechi_iheanacho,-102.0
320,adrian_san_miguel_del_castillo,-100.0


In [39]:
combined.sort_values('rank_increase', ascending=False)[['name', 'rank_increase']].head(20)

,name,rank_increase
377,jeff_hendrick,280.0
411,karl_darlow,223.0
396,grady_diangana,223.0
336,bernd_leno,214.0
400,steven_alzate,174.0
406,callum_robinson,159.0
334,n'golo_kante,125.0
252,pierre-emile_hojbjerg,121.0
380,moussa_djenepo,116.0
198,hector_bellerin,114.0
